<h1>Методы борьбы с несбалансированными классами</h1>

Часто, полученные нами данные оказываются несбалансированными, то есть объектов одного класса оказывается в несколько раз больше чем другого(к примеру, 90% и 10%).Большинство алгоритмов машинного обучения хорошо работают когда количество представителей каждого класса примерно равны.Это потому что большинство алгоритмов заточено на максимизацию точности (accuracy) и минимизацию ошибок.Для корректной работы важно правильно настроить методы и метрики, чтобы достигнуть желыемых целей.

К примеру, в используемом нами датасете мы заинтересованы в выявлении мошеннических операций по карте, которые составляют всего 0,17% от всего датасета. Если в этом случае мы будем использовать accuracy в качестве основной метрики, то получим отличный, но бесполезный на практике результат - 99,8% что говорит лишь о качестве выявления мажорируемого класса - неинтересующие нас немошеннические операции.



Поэтому для несбалансированных данных предпочтительнее использовать такие метрики как <b>precision<b>, <b>recall<b>, и <b>f1-score<b>

<h2>Как справиться с несбалансированным датасетом? </h2>

<ul>
  <li><h4>Under-Sampling</h4>  Уменьшить количество представителей мажорного класса </li>  
  <li><h4>Over-Sampling</h4>    Добавить копий представителей минорного класса</li> 
  <li><h4>SMOTE</h4> Искусственно синтезировать экземпляры минорного класса    </li>
  <li><h4>Change the algorithm</h4>Воспользоваться другим алгоритмом классификации    </li>
    
</ul>

<h3> Исходный датасет и LogRegression </h3>

In [21]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score


creditcard = pd.read_csv('creditcard.csv')
y = creditcard.Class
X = creditcard.drop('Class', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)

lr = LogisticRegression(solver='liblinear').fit(X_train, y_train)

lr_pred = lr.predict(X_test)
#accuracy_score(y_test, lr_pred)
#recall_score(y_test, lr_pred)
#f1_score(y_test, lr_pred)



|accuracy | recall | f1-score |
|---------|--------|----------|
|0,999    |0,644   |0,724     |

<h3>Over-Sampling minority class</h3>

In [16]:
from sklearn.utils import resample


creditcard = pd.read_csv('creditcard.csv')
y = creditcard.Class
X = creditcard.drop('Class', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)

#соединим тренировочный сет обратно
X = pd.concat([X_train, y_train], axis=1)

#выделим классы fraud и non-fraud
not_fraud = X[X.Class==0]
fraud = X[X.Class==1]

#увеличиваем количество представителей минорного класса
fraud_upsampled = resample(fraud,replace=True,n_samples=len(not_fraud),random_state=27)

#соединяеям все в один датасет
upsampled = pd.concat([not_fraud, fraud_upsampled])

y_train = upsampled.Class
X_train = upsampled.drop('Class', axis=1)

upsampled = LogisticRegression(solver='liblinear').fit(X_train, y_train)

upsampled_pred = upsampled.predict(X_test)

#accuracy_score(y_test, upsampled_pred)
#recall_score(y_test, upsamled_pred)
#f1_score(y_test, upsampled_pred)





0.14375000000000002

|accuracy | recall | f1-score | dataset |
|---------|--------|----------|---------|
|0,999    |0,644   |0,724     | original|
|0,980    |0,871   |0,143     |over     |

Не самый лучший вариант, потому что recall увеличилась, но F1 мера сильно упала

<h3>Under-Sampling majority class</h3>

In [22]:
creditcard = pd.read_csv('creditcard.csv')
y = creditcard.Class
X = creditcard.drop('Class', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)

#выделяем тренировочный сет
X = pd.concat([X_train, y_train], axis=1)

not_fraud = X[X.Class==0]
fraud = X[X.Class==1]

#уменьшаем количество мажорного класса
not_fraud_downsampled = resample(not_fraud,replace = False, n_samples = len(fraud),random_state = 27)

downsampled = pd.concat([not_fraud_downsampled, fraud])

y_train = downsampled.Class
X_train = downsampled.drop('Class', axis=1)

undersampled = LogisticRegression(solver='liblinear').fit(X_train, y_train)

undersampled_pred = undersampled.predict(X_test)



|accuracy | recall | f1-score | dataset |
|---------|--------|----------|---------|
|0,999    |0,644   |0,724     | original|
|0,975    |0,863   |0,117     |under    |

Так же не лучший вариант

Еще один способ балансировки данных - искусственное их синтезирование <b>SMOTE(Synthetic Minority Oversampling Technique).</b>
    При таком подходе мы искусственно добавляем экземпляры минорного класса по методу ближайших соседей.

<h3>SMOTE</h3>

In [26]:
import imblearn
from imblearn.over_sampling import SMOTE

y = creditcard.Class
X = creditcard.drop('Class', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)

sm = SMOTE(random_state=27)
X_train, y_train = sm.fit_sample(X_train, y_train)

smote = LogisticRegression(solver='liblinear').fit(X_train, y_train)
smote_pred = smote.predict(X_test)
f1_score(y_test, smote_pred)

0.18461538461538463

|accuracy | recall | f1-score | dataset |
|---------|--------|----------|---------|
|0,999    |0,644   |0,724     | original|
|0,985    |0,863   |0,184     |SMOTE    |

Результат схож с предыдущим вариантом

Мы можем попоробовать так же изменить сам алгоритм и вместо логистической регрессии использовать, к примеру, решающие деревья

<h3>Random Forest Classifier </h3>

In [30]:
from sklearn.ensemble import RandomForestClassifier

y = creditcard.Class
X = creditcard.drop('Class', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)

rfc = RandomForestClassifier(n_estimators=10).fit(X_train, y_train)
rfc_pred = rfc.predict(X_test)



|accuracy | recall | f1-score | dataset |
|---------|--------|----------|---------|
|0,999    |0,644   |0,724     | original|
|0,999    |0,788   |0,874     |RFC      |

В этом случае мы видим прирост по всем метрикам и можем сделать вывод, что решающие деревья - один из лучших алгоритмов для этого датасета.
